In [1]:
import time
import json
from pprint import pprint
from sqlalchemy import create_engine, Column, MetaData, func, NUMERIC, VARCHAR, BOOLEAN
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from python_opendota.apis.tags import benchmarks_api
import python_opendota
import requests

In [ ]:
# Настройки подключения к базе данных

In [2]:
POSTGRES_USER = "petrakovpk"
POSTGRES_PASSWORD = "Ef9iei!!"
POSTGRES_SERVER = "134.0.112.69"
POSTGRES_PORT = "5432"
POSTGRES_DB = "dota2"

DATABASE_URL = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_SERVER}:{POSTGRES_PORT}/{POSTGRES_DB}"

engine = create_engine(DATABASE_URL)

Session = sessionmaker(bind=engine)
session = Session()

In [3]:
# Создаем нашу модель
Base = declarative_base()

class MatchInDb(Base):
    __tablename__ = "matches"

    match_id = Column(NUMERIC, primary_key=True)
    duration = Column(NUMERIC)
    start_time = Column(NUMERIC)
    radiant_team_id = Column(NUMERIC)
    radiant_name = Column(VARCHAR)
    dire_team_id = Column(NUMERIC)
    dire_name = Column(VARCHAR)
    leagueid = Column(NUMERIC)    
    league_name = Column(VARCHAR)
    series_id = Column(NUMERIC)
    series_type = Column(NUMERIC)
    radiant_score = Column(NUMERIC)
    dire_score = Column(NUMERIC)
    radiant_win = Column(BOOLEAN)
    
class TeamInDb(Base):
    __tablename__ = "teams"
    
    team_id = Column(NUMERIC, primary_key=True)
    rating = Column(NUMERIC)
    wins = Column(NUMERIC)
    losses = Column(NUMERIC)
    last_match_time = Column(NUMERIC)
    name = Column(VARCHAR)
    tag = Column(VARCHAR)
    logo_url = Column(VARCHAR)

In [4]:
# Объявляем переменные для request
base_url = "http://api.opendota.com/api"

teams_url = base_url + "/teams"
pro_matches_url = base_url + "/proMatches"

In [7]:
number_of_matches_to_add = 10000
number_of_matches_in_request = 100

iterations = int(number_of_matches_to_add / number_of_matches_in_request)

for i in range(iterations):
    less_than_match_id = session.query(func.min(MatchInDb.match_id)).first()[0]
    pro_matches_url = base_url + "/proMatches" + "?less_than_match_id=" + str(less_than_match_id)

    pro_matches_result = requests.get(pro_matches_url)
    pro_matches_result_json = pro_matches_result.json()

    if "less_than_match_id" in pro_matches_result_json:
        pro_matches = pro_matches_result_json["less_than_match_id"]

        # Сохраните все матчи в одной транзакции
        matches = [MatchInDb(**pro_match_json) for pro_match_json in pro_matches]
        session.add_all(matches)
        session.commit()

    time.sleep(1)

    print("Итерация завершена: ", i)
        
    i -= 1

Итерация завершена:  0
Итерация завершена:  1
Итерация завершена:  2
Итерация завершена:  3
Итерация завершена:  4
Итерация завершена:  5
Итерация завершена:  6
Итерация завершена:  7
Итерация завершена:  8
Итерация завершена:  9
Итерация завершена:  10
Итерация завершена:  11
Итерация завершена:  12
Итерация завершена:  13
Итерация завершена:  14
Итерация завершена:  15
Итерация завершена:  16
Итерация завершена:  17
Итерация завершена:  18
Итерация завершена:  19
Итерация завершена:  20
Итерация завершена:  21
Итерация завершена:  22
Итерация завершена:  23
Итерация завершена:  24
Итерация завершена:  25
Итерация завершена:  26
Итерация завершена:  27
Итерация завершена:  28
Итерация завершена:  29
Итерация завершена:  30
Итерация завершена:  31
Итерация завершена:  32
Итерация завершена:  33
Итерация завершена:  34
Итерация завершена:  35
Итерация завершена:  36
Итерация завершена:  37
Итерация завершена:  38
Итерация завершена:  39
Итерация завершена:  40
Итерация завершена:  41
Ит

In [9]:
# Получаем список команд
team_ids_query = session.query(MatchInDb.radiant_team_id.label("team_id"))\
.outerjoin(TeamInDb, MatchInDb.radiant_team_id == TeamInDb.team_id)\
.filter(TeamInDb.team_id == None)\
.filter(MatchInDb.radiant_team_id != None)

team_ids_query = team_ids_query.union_all(
    session.query(MatchInDb.dire_team_id.label("team_id"))\
    .outerjoin(TeamInDb, MatchInDb.dire_team_id == TeamInDb.team_id)\
    .filter(TeamInDb.team_id == None)\
    .filter(MatchInDb.dire_team_id != None)
)

# Используем subquery для создания подзапроса
team_ids_subquery = team_ids_query.subquery()

# Выполняем основной запрос
team_ids = session.query(team_ids_subquery.c.team_id).all()


In [10]:
# Переменная для задержки
delay = 1

for team_id in team_ids:
    # Получаем URL
    team_id = team_id[0]
    team_url = teams_url + "/" + str(team_id)

    # Получаем информацию по команде
    team_result = requests.get(team_url)

    # Превращаем их в JSON
    team_result_json = json.loads(team_result.text)

    # Сохраняем в БД
    try:
        team_data = {key: value for key, value in team_result_json.items()}
        team = TeamInDb(**team_data)
        session.add(team)
        session.commit()
        session.refresh(team)

    except Exception as e:
        print(f"Ошибка при сохранении команды: {str(e)}")
        continue

    # Засыпаем
    time.sleep(delay)